<a href="https://colab.research.google.com/github/AJLR888/hmda-ny-2007-loan-default/blob/main/ny_2007_data_preprocesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HMDA. Mortgage Data Analysis and Modeling Predictions.**

## The Dataset: https://www.consumerfinance.gov/data-research/hmda/

The Home Mortgage Disclosure Act (HMDA) requires many financial institutions to maintain, report, and publicly disclose loan-level information about mortgages. These data help show whether lenders are serving the housing needs of their communities; they give public officials information that helps them make decisions and policies; and they shed light on lending patterns that could be discriminatory. The public data are modified to protect applicant and borrower privacy.

HMDA was originally enacted by Congress in 1975 and is implemented by Regulation C.

## The Goal:
We will use the models Logistic Regression and LightGBM. to assess the outcomes, we will use ROC-AUC and Log Loss. The purpouse will be:

1.   **Find out if the are any intersectional biases** in model's predictions.
2.   **Which features influenced the most on model's outcomes**. We will use SHAP Values.

## Steps:

1.   Setup of working environment and libraries.
2.   Overview:
  *   A) Understanding features: Know what each column represents and whether it is useful.
  *   B) Understanding the data, shape and structure:

      *   df.shape
      *   df.columns
      *   df.dtypes
      *   df.head()
3.   Data cleaning:
  *   A) Excluding irrelevant records from columns.
      * Quality issues: in "edit_status_name" there are 190,210 records with quality issues. We will ommit them.
      *  Only idividuals: we will focus on individuals hence from 'co_applicant_ethnicity_name' we only keep 'No co-applicant'

  *   B) Excluding irrelevant columns:
  
  *   C) Excluding irrelevant records from:
      *   applicant_ethnicity_name
      *   applicant_race_name_1
      *   applicant_sex_name
      *   action_taken_name
      
  *   D) Addressing missing values

4.   Exploratory Data Analysis
  *   A) ds
5.   Feature engineering
  *   A) Feature creation

  *   B) Feature





# Step 1: Setup of working environment and libraries.




Note:

 *   !git init is for Google Colab environment, we should use "subprocess.run(["git", "init"])" which is more in line with PEP 8 doc. Style.

 *   We only use once "git init", when starting a new project. The purpose of this code is to create things related to the project in a .git directory in the project folder.

 *   subprocess.run(["git", "init"])

In [47]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
# Setting up GitHub
import os
import subprocess
from getpass import getpass #Secure token storage

# Importing working space
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [49]:
# Username and email:
subprocess.run(["git", "config", "--global", "user.name", "AJLR888"], check=True)
subprocess.run(["git", "config", "--global", "user.email", "roldan.analytics@gmail.com"], check=True)


# Storing GitHub token and repository details
GITHUB_TOKEN = getpass("Enter GitHub Token:")
REPO_OWNER = "AJLR888"
REPO_NAME = "hmda-ny-2007-loan-default"
BRANCH_NAME = "main"

#Setting GitHub remot URL with authentcation
GIT_REMOTE_URL = f"https://{GITHUB_TOKEN}@github.com/{REPO_OWNER}/{REPO_NAME}.git"
os.system(f"git remote set-url origin {GIT_REMOTE_URL}")

Enter GitHub Token:··········


0

In [50]:
# Loading the data
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/hmda_2007_ny_all-records_labels.csv')

# Checking data size
df = pd.DataFrame(df)


<ipython-input-50-100f7bdf4485>:2: DtypeWarning: Columns (34,36,38,42,44,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/hmda_2007_ny_all-records_labels.csv')


# Step 2: General overview



## A)   Understanding of each feature. What does each feature describe?




1.   **as_of_year**: The year the mortgage was reported.
2.   **respondent_id**: The lender.
3.   **agency_name**: The regulatory agency responsible for overseeing the financial institution that reported the mortgage.
4.   **agency_abbr**: Agency abrebiation name.

5.   **agency_code**: The regulator that supervises the regulated lender that report the data.

6.   **loan_type_name**:The type of covered loan or application (if covered). By covered mean, there is a mechanism to protect the lender.

7.   **property_type_name**:

  *   One to four-family (other than manufactured housing)
  *   Manufactured housing
  *   Multifamily


8.   **loan_purpose_name**: What was the loan used for.

9.   **owner_occupancy_name**: Whether the owner intend to live in the property or not.

10.   **loan_amount_000s**

11.   **preapproval_name**: Preapproval only applies to home purchase loans, not refinancing or home improvement loans.

12.   **action_taken_name**: The final outcome of a loan application.

13.   **msamd_name**: MSA stands for Metropolitan Statistical Area, is a region with a high population density at its core and close economic ties throught the area; MD stands for Metropolitan division, is a sub-region within a large MSA (used when the area is particularly populous).


  *   *Considerations*:
    *   If the property is outside of the MSA/MD area it may be blank/empty.
    *   Helps analyze lending patterns, approval rates, and borrower demographics in specific regions.
    *   msamd is broader, city or regional-level in comparison with census_tract_number. msamd is better use for housing trends or market segmentation for instance.

14.   **census_tract_number**: is a small area. Designed to be socially and economically homogeneous. In other words, they are kind of neighbourhoods.

16.   **purchaser_type_name**: Describes who purchased the loan in the secondary market after it was originated by the reporting institution.

17.   **denial_reason_name_1**: Important to consider that if NA the loan was not denied.

18.   **rate_spread:** The exra interest that a borrower pay compared to the "best-qualified" borrowers. Generally speaking it means higher risk although there are nuances to consider. e.g. someone can have a higher rate spread due to a bad credit score which doesn't reflect their ability and will to pay back.

19.   **hoepa_status_name**: hoepa stands for Home Ownership and Equity Protection Act. is a law disgned to protect borrowers from predatory lending.
  *   Considerations:
    *   It can be used to track if high-cost loans are disproportionally targeted at vulnerable groups.

20.   **lien_status_name**: Basically, it indicates whether the mortgage is secured or not.

21.   **edit_status_name**: Are rules to assist filers in checking the accuracy of HMDA data prior to submission. Idicate if there are any accuracy(quality)issues.  
    *   Note: In our dataset we have almost 200,000 records as **"Quality edit failure only"** which is indicative of some potentail data inconsistency.

22.   **minority_population:** % of minority population to total tract population.

23.   **hud_median_family_income:** Median family income for the MSA (metropolitan statistical area) or MD (metropolitan division)
      *   Note: hud(Housing & Urban development).
*   List item

24.   **population**: total population in tract (tract ~ neighbourhood).

25.   **tract_to_msamd_income:** Indicates how wealthy or poor a tract is compared to its surroundings areas. compares the median family income of the census tract to the median family income of the corresponding metropolitan statistical area MSA OR metropolitan division MD. Let's remember that a tract is inside of a MSA/MD.

25.   **number_of_owner_occupied_units:** Total number of housing units in the census tract that are occupied by their owners rather than rented.

26.   **number_of_1_to_4_family_units:** Indicates the number of housing units that are classified as single-family homes. The higher the number the less population density.

--------------------------------------------------------------------------------

Check the following sources for further undertanding:

* Check filing instructions guide for further information: https://ffiec.cfpb.gov/documentation/fig/2024/overview

* Glosary: https://www.ffiec.gov/hmda/glossary.htm#top

* Original source: https://www.consumerfinance.gov/data-research/hmda/historic-data/?geo=nationwide&records=all-records&field_descriptions=labels

* Check "lar_record_codes.pdf" from my GitHub for further Inf.

##  B) Understanding the data

In [51]:
# Checking content in the columns
pd.set_option('display.max_columns', None)
print(df.head(5))

   as_of_year respondent_id                                  agency_name  \
0        2007    20-2096530    Office of the Comptroller of the Currency   
1        2007    2085300005  Department of Housing and Urban Development   
2        2007    20-2096530    Office of the Comptroller of the Currency   
3        2007    0000003970                 Office of Thrift Supervision   
4        2007    2085300005  Department of Housing and Urban Development   

  agency_abbr  agency_code loan_type_name  loan_type  \
0         OCC            1   Conventional          1   
1         HUD            7   Conventional          1   
2         OCC            1   Conventional          1   
3         OTS            4   Conventional          1   
4         HUD            7   Conventional          1   

                                  property_type_name  property_type  \
0  One-to-four family dwelling (other than manufa...              1   
1  One-to-four family dwelling (other than manufa...            

In [52]:
# Force to dislplay all rows
pd.set_option('display.max_rows', None)

# Data types
print(df.dtypes)

as_of_year                          int64
respondent_id                      object
agency_name                        object
agency_abbr                        object
agency_code                         int64
loan_type_name                     object
loan_type                           int64
property_type_name                 object
property_type                       int64
loan_purpose_name                  object
loan_purpose                        int64
owner_occupancy_name               object
owner_occupancy                     int64
loan_amount_000s                    int64
preapproval_name                   object
preapproval                         int64
action_taken_name                  object
action_taken                        int64
msamd_name                         object
msamd                             float64
state_name                         object
state_abbr                         object
state_code                          int64
county_name                       

In [53]:
# Null values
print(df.isnull().sum())

as_of_year                              0
respondent_id                           0
agency_name                             0
agency_abbr                             0
agency_code                             0
loan_type_name                          0
loan_type                               0
property_type_name                      0
property_type                           0
loan_purpose_name                       0
loan_purpose                            0
owner_occupancy_name                    0
owner_occupancy                         0
loan_amount_000s                        0
preapproval_name                        0
preapproval                             0
action_taken_name                       0
action_taken                            0
msamd_name                          92920
msamd                               92920
state_name                              0
state_abbr                              0
state_code                              0
county_name                       

In [74]:
# Unique values for categorical variables
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
  print(f"Value counts for: {col}")
  print(df[col].value_counts(), "\n")
  print("\n" + "=+"*50 + "\n")

Value counts for: loan_type_name
loan_type_name
Conventional          304712
FHA-insured            12637
VA-guaranteed           1211
FSA/RHS-guaranteed       281
Name: count, dtype: int64 


=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+

Value counts for: property_type_name
property_type_name
One-to-four family dwelling (other than manufactured housing)    313554
Manufactured housing                                               5287
Name: count, dtype: int64 


=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+

Value counts for: loan_purpose_name
loan_purpose_name
Home purchase       138880
Refinancing         133651
Home improvement     46310
Name: count, dtype: int64 


=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+

Value counts for: action_taken_name
action_taken_name
Loan originated                                148863


# Step 3: Data cleaning

## C) Excluding irrelevant records from columns

In [55]:
# Records/columns
df.shape

(1009451, 78)

In [56]:
# Quality issues: There are 190210 records with quality issues. We will ommit them.
df = df[~df["edit_status_name"].isin(["Quality edit failure only"])]

# Our focus is on individuals hence we will focus on data with "No co-applicant"
df = df[df['co_applicant_ethnicity_name'] == 'No co-applicant']

In [57]:
# Records/shape
df.shape

(472701, 78)

## E) Excluding irrelevant records from the following columns:

*   applicant_ethnicity_name
*   applicant_race_name_1
*   applicant_sex_name
*   action_taken_name

In [58]:
df = df[
    ~df["action_taken_name"].isin([
        "File closed for incompleteness",
        "Preapproval request denied by financial institution",
        "Preapproval request approved but not accepted",
        "Application withdrawn by applicant"
    ]) &
    ~df["applicant_ethnicity_name"].isin([
        "Information not provided by applicant in mail, Internet, or telephone application",
        "Not applicable"
    ]) &
    ~df["applicant_race_name_1"].isin([
        "Information not provided by applicant in mail, Internet, or telephone application",
        "Not applicable"
    ]) &
    ~df["applicant_sex_name"].isin([
        "Information not provided by applicant in mail, Internet, or telephone application",
        "Not applicable"
    ])
]


In [59]:
print(df.shape)

(323044, 78)


## F) Addressing missing values

In [60]:
print(df.isnull().sum())

as_of_year                             0
respondent_id                          0
agency_name                            0
agency_abbr                            0
agency_code                            0
loan_type_name                         0
loan_type                              0
property_type_name                     0
property_type                          0
loan_purpose_name                      0
loan_purpose                           0
owner_occupancy_name                   0
owner_occupancy                        0
loan_amount_000s                       0
preapproval_name                       0
preapproval                            0
action_taken_name                      0
action_taken                           0
msamd_name                         34069
msamd                              34069
state_name                             0
state_abbr                             0
state_code                             0
county_name                          357
county_code     

### Filling and dropping

In [61]:
df = df.assign(
    msamd_name=df['msamd_name'].fillna("Unknown"),
    denial_reason_name_1=df['denial_reason_name_1'].fillna("Unknown"),
    rate_spread=df['rate_spread'].fillna(0)
)

df = df.dropna(subset=['county_name',
                       'census_tract_number',
                       'applicant_income_000s',
                       'minority_population',
                       'hud_median_family_income',
                       'tract_to_msamd_income',
                       'number_of_owner_occupied_units',
                       'number_of_1_to_4_family_units'
                       ]
               )


In [62]:
print(df.isnull().sum())

as_of_year                             0
respondent_id                          0
agency_name                            0
agency_abbr                            0
agency_code                            0
loan_type_name                         0
loan_type                              0
property_type_name                     0
property_type                          0
loan_purpose_name                      0
loan_purpose                           0
owner_occupancy_name                   0
owner_occupancy                        0
loan_amount_000s                       0
preapproval_name                       0
preapproval                            0
action_taken_name                      0
action_taken                           0
msamd_name                             0
msamd                              33468
state_name                             0
state_abbr                             0
state_code                             0
county_name                            0
county_code     

# Step 4: Exploratory Data Analysis

# Step 5: Feature engineering

## A) Feature creation


In [76]:
# Creation of loan_to_income_ratio column:
df['loan_to_income_ratio'] = (df['loan_amount_000s'] / df['applicant_income_000s']).round(2)

# Checking
print(df['loan_to_income_ratio'].head())

# Note: Figures below represent the times of their annual income.
# E.g. 3.10, means that the loan amount is 3.10 times their annual income.

0    3.38
2    0.85
5    3.07
6    1.76
7    1.33
Name: loan_to_income_ratio, dtype: float64


In [64]:
# Creation of ethnicity_race_sex column:
df['ethnicity_race_sex'] = df['applicant_ethnicity_name'].str.lower() + "_" + df['applicant_race_name_1'].str.lower() + "_" + df['applicant_sex_name'].str.lower()

# Checking column created
print(df[['ethnicity_race_sex']].value_counts())

ethnicity_race_sex                                                     
not hispanic or latino_white_male                                          126540
not hispanic or latino_white_female                                         84557
not hispanic or latino_black or african american_female                     27181
not hispanic or latino_black or african american_male                       22053
hispanic or latino_white_male                                               17879
not hispanic or latino_asian_male                                           13608
hispanic or latino_white_female                                             11132
not hispanic or latino_asian_female                                          8975
not hispanic or latino_american indian or alaska native_male                 1116
hispanic or latino_black or african american_male                             937
not hispanic or latino_native hawaiian or other pacific islander_male         922
hispanic or latino_black o

## B) Feature selection

In [66]:
df = df[[
    "loan_type_name",
    "property_type_name",
    "loan_purpose_name",
    "loan_amount_000s",
    "action_taken_name",
    "msamd_name",
    "census_tract_number",
    "hoepa_status_name",
    "applicant_ethnicity_name",
    "applicant_race_name_1",
    "applicant_sex_name",
    "applicant_income_000s",
    "denial_reason_name_1",
    "rate_spread",
    "lien_status_name",
    "hud_median_family_income",
    "tract_to_msamd_income",
    "minority_population",
    "number_of_owner_occupied_units"
]]

# Commit to GitHub


The code below will help us to commit our project onto GitHub:



In [67]:
os.chdir("/content/drive/My Drive/Colab Notebooks/hmda_ny_2007_preprocessing/") #First we call our project's location using we use os.chdir() instead of %cd because
                                                                                #the first option is the better choice for code that needs to be portable and run in different environments
                                                                                #and we are trying to replicate a working environment.

#Source: https://www.tutorialspoint.com/python/os_chdir.htm

In [68]:
#We use subprocess.run(["git", "status"]) instead of "!git status" (Google Colab) for a PEP-8.
result = subprocess.run(["git", "status"], capture_output=True, text=True) # The following helps us to "capture the results"; capture_output=True, text=True

print(result.stdout)  # Print the status message
print(result.stderr)  # Print errors if there are any.

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ny-2007-data-preprocesing.ipynb

no changes added to commit (use "git add" and/or "git commit -a")




In [69]:
subprocess.run(["git", "add", "."], check=True) #!git add . |/| We use "check=True" to ensure that if there are any error, they will be shown.

CompletedProcess(args=['git', 'add', '.'], returncode=0)

In [70]:
subprocess.run(["git", "commit", "-m", "Updating-20:00-25032025"], check=True) #Google Colab: !git commit -m "Updating"

CompletedProcess(args=['git', 'commit', '-m', 'Updating-20:00-25032025'], returncode=0)

In [71]:
# ERROR, NEED TO FIX: subprocess.run(["git", "push", "origin", "master"], check=True) #!git push origin main #gave error, solution !git config --global credential.helper store

In [72]:
subprocess.run(args=['git', 'branch'], check=True)


CompletedProcess(args=['git', 'branch'], returncode=0)

In [73]:
logStatus = subprocess.run(["git", "log"], capture_output=True, text=True, check=True) #Google Colab: !git log
print(logStatus.stdout) #Shows output
print(logStatus.stderr) #Shows errors

commit b634770840aeef42541923b3a696df643704102c
Author: AJLR888 <roldan.analytics@gmail.com>
Date:   Sun Mar 30 10:19:59 2025 +0000

    Updating-20:00-25032025

commit 8ae74f3ea231ed1cc166a47a3ce92cc78cb2469d
Author: AJLR888 <roldan.analytics@gmail.com>
Date:   Sun Mar 30 09:43:33 2025 +0000

    Updating-20:00-25032025

commit 22f14bb8dd2143947c156433309f96c7048974f4
Author: AJLR888 <roldan.analytics@gmail.com>
Date:   Mon Mar 24 20:21:10 2025 +0000

    Updating-20:19-24032025

commit c7fad8581585547c97e2d4d0653c80278f40d5b0
Author: AJLR888 <roldan.analytics@gmail.com>
Date:   Mon Mar 24 17:57:19 2025 +0000

    Updating-08:40-21032025

commit 3c9d2766364bfb3617c9a311742c26f18461db5e
Author: AJLR888 <roldan.analytics@gmail.com>
Date:   Sun Mar 16 10:21:19 2025 +0000

    Updating10:20-16032025

commit 7a277871e6a4326cefb4c4e12992b04615fdbb6d
Author: AJLR888 <roldan.analytics@gmail.com>
Date:   Sun Mar 16 10:19:04 2025 +0000

    Updating1412-15032025

commit c75bc5b5e448ae2ddf6b9a1b